In [36]:
def load_pickle(data: str):
    with open(data, 'rb') as f:
        ret = pickle.load(f)
    return ret


In [37]:
import sqlite3 as sq
import pickle
conn = sq.connect("../crowd.sqlite")

In [94]:
create_table = lambda table, params: f"create table IF NOT EXISTS {table} (ID INTEGER PRIMARY KEY AUTOINCREMENT, boxsize integer, q float, {params})"

insert = lambda table, params: f"insert into {table} values ({params})"

In [39]:
crowd_brownian_fits = load_pickle("crowd_brownian_fit.pickle")
crowd_ballistic_fits = load_pickle("crowd_ballistic_fit.pickle")
running_brownian_fits = load_pickle("running_brownian_fit.pickle")
running_ballistic_fits = load_pickle("running_ballistic_fit.pickle")

In [109]:
def add_to_db(conn, table, isf: str):
    params = f"tauc float, tauc_err float, {' v float, v_err float,' if isf == 'ballistic' else ''} A float, A_err float, B float, B_err float"
    for video_name, v in table.items():
        table_name = f"fit_{isf}_{video_name}"
#         with conn:
#                 conn.execute(f"DROP TABLE IF EXISTS {table_name}")
        with conn:
            conn.execute(create_table(table_name, params))
        for box_size, v1 in v.items():
            tau = v1[0]
            data = v1[1]
            for q, fits_errs in data.items():
                fits, errs = fits_errs
                k = ", ".join(map(lambda x: f"{x[0]}, {x[1]}", zip(fits, errs)))
                values = f"null, {box_size}, {q}, {k}"
                with conn:
                    conn.execute(insert(table_name, values))


In [110]:
add_to_db(conn, crowd_ballistic_fits, "ballistic")
add_to_db(conn, crowd_brownian_fits, "brownian")
add_to_db(conn, running_ballistic_fits, "ballistic")
add_to_db(conn, running_brownian_fits, "brownian")